<h1 align='center'>
 <b>ETL</b>
</h1>


Como paso inicial, se llevó a cabo la apertura y exploración de los datos utilizando las funciones y herramientas disponibles en Google Cloud Platform. Este proceso permitió acceder a los datos y realizar un análisis preliminar para comprender la naturaleza y la estructura de la información con la que estábamos trabajando.

### Importaciones

In [ ]:
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
import os 
import ast
import json

Se recomienda leer el archivo [diccionario de datos](Diccionario_De_Datos.md) para una comprensión de los datos iniciales proporcionados

### Review Por Estados

Se procede a realizar una  función que facilita la lectura y procesamiento de archivos JSON dentro de una carpeta y sus subcarpetas, convirtiéndolos en un DataFrame de pandas y guardándolos como archivos CSV para todos los review sobre estados que nos proporcionaron

In [ ]:
def leer_contenido_carpeta(ruta_carpeta):
    """
    Lee el contenido de una carpeta y devuelve una lista de diccionarios con los datos de los archivos JSON.

    Args:
        ruta_carpeta (str): Ruta de la carpeta a leer.

    Returns:
        list: Lista de diccionarios con los datos de los archivos JSON.
    """
    
    try:
        # Verificar si la ruta es un directorio
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            contenido = os.listdir(ruta_carpeta)
            
            # Lista para almacenar datos de todos los archivos JSON en la carpeta
            df = []
            
            # Recorrer la lista de archivos y directorios
            for item in contenido:
                item_ruta = os.path.join(ruta_carpeta, item)
                
                # Si es un archivo, leer si es JSON
                if os.path.isfile(item_ruta) and item.endswith('.json'):
                    print("Leyendo el archivo:", item_ruta)
                    
                    # Leer y procesar el archivo JSON
                    with open(item_ruta, 'r', encoding='utf-8') as f:
                        for line in f:
                            df.append(json.loads(line))
                
                # Si es un directorio, llamar recursivamente a la función
                elif os.path.isdir(item_ruta):
                    print(f"Entrando en carpeta: {item_ruta}")
                    subcontenido = leer_contenido_carpeta(item_ruta)
                    if subcontenido is not None:
                        df.extend(subcontenido)
            
            # Convertir la lista de diccionarios a DataFrame
            df_final = pd.DataFrame(df)
            
            # Especificar la ruta donde se guardarán los archivos CSV
            ruta_guardado = '../csv/Google Maps/reviews-estados'
            
            # Obtener el nombre de la carpeta que contiene los archivos JSON
            nombre_carpeta = os.path.basename(ruta_carpeta)
            
            # Guardar el DataFrame como CSV en la ruta especificada
            nombre_archivo_final = os.path.join(ruta_guardado, nombre_carpeta + '.csv')
            df_final.to_csv(nombre_archivo_final, index=False)
            print("Se guardó con éxito en:", nombre_archivo_final)                    
                    
        else:
            print(f"La ruta '{ruta_carpeta}' no es un directorio válido.")
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")

In [ ]:
leer_contenido_carpeta('../Data/Google Maps/reviews-estados/')

### Metadata

Esa crea una función que procesa los archivos JSON de la carpeta metadata-sitios, crea un DataFrame con los datos procesados y, finalmente, guarda ese DataFrame como un archivo CSV

In [ ]:
import os
def leer_contenido_carpeta(ruta_carpeta):
    """ Lee el contenido de una carpeta y devuelve una lista de archivos y directorios.

    Args:
        ruta_carpeta (str): Ruta de la carpeta a leer.

    Returns:
        list: Lista de archivos y directorios.
    """
    
    try:
        # Verificar si la ruta es un directorio
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            contenido = os.listdir(ruta_carpeta)
            
            # Lista para almacenar datos de todos los archivos JSON en la carpeta
            df = []
            
            # Recorrer la lista de archivos y directorios
            for item in contenido:
                item_ruta = os.path.join(ruta_carpeta, item)
                
                # Si es un archivo, leer si es JSON
                if os.path.isfile(item_ruta) and item.endswith('.json'):
                    print("Leyendo el archivo:", item_ruta)
                    
                    # Leer y procesar el archivo JSON
                    with open(item_ruta, 'r', encoding='utf-8') as f:
                        for line in f:
                            df.append(json.loads(line))
            
            # Convertir la lista de diccionarios a DataFrame
            df_final = pd.DataFrame(df)
            
            # Especificar la ruta donde se guardarán los archivos CSV
            ruta_guardado = '../csv/Google Maps/metadata-sitios'
            
            # Obtener el nombre de la carpeta que contiene los archivos JSON
            nombre_carpeta = os.path.basename(ruta_carpeta)
            
            # Guardar el DataFrame como CSV en la ruta especificada
            nombre_archivo_final = os.path.join(ruta_guardado, nombre_carpeta + '.csv')
            df_final.to_csv(nombre_archivo_final, index=False)
            print("Se guardó con éxito en:", nombre_archivo_final)                    
                    
        else:
            print(f"La ruta '{ruta_carpeta}' no es un directorio válido.")
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")


In [ ]:
leer_contenido_carpeta('../Data/Google Maps/metadata-sitios/')

### Yelp User Parquet dividido

In [ ]:
def dividir_archivo_parquet_en_csv(ruta_archivo_parquet, carpeta_destino_csv, num_filas_por_archivo=450000):
    """ Divide un archivo Parquet en partes más pequeñas y las guarda como CSV.
    El archivo Parquet se lee y se divide en partes más pequeñas. Cada parte se guarda como un archivo CSV.
    El número de filas por archivo se puede establecer como un parámetro opcional.
    Por defecto, el número de filas por archivo es de 450000.
    El archivo CSV se guarda en la carpeta especificada como parámetro.
    Si la carpeta no existe, se crea.
    Si el archivo Parquet no existe, se levanta una excepción.
    Si el archivo Parquet está vacío, se levanta una excepción.
    Si el archivo Parquet tiene menos filas que el número de filas por archivo, se levanta una excepción.
    """
    
    # Leer el archivo Parquet
    archivo_grande = pd.read_parquet(ruta_archivo_parquet)

    # Dividir el archivo en partes más pequeñas
    num_archivos = (len(archivo_grande) // num_filas_por_archivo) + 1

    for i in range(num_archivos):
        inicio = i * num_filas_por_archivo
        fin = min((i + 1) * num_filas_por_archivo, len(archivo_grande))  
        archivo_pequeno = archivo_grande.iloc[inicio:fin]
        
        # Guardar el archivo como CSV
        nombre_archivo_csv = f'archivo_pequeno_{i}.csv'
        ruta_completa_csv = os.path.join(carpeta_destino_csv, nombre_archivo_csv)
        archivo_pequeno.to_csv(ruta_completa_csv, index=False)

In [ ]:
ruta_archivo_grande = '../Data/Yelp/user.parquet'
carpeta_archivos_pequenos_csv = '../csv/Yelp/user/'
dividir_archivo_parquet_en_csv(ruta_archivo_grande, carpeta_archivos_pequenos_csv)

### Yelp Json

In [ ]:
def procesar_archivo_json(ruta_archivo_json, ruta_guardado):
    
    """ 
    Procesa un archivo JSON y lo guarda en un archivo CSV.
    Args:
        ruta_archivo_json (str): Ruta del archivo JSON a procesar.
        ruta_guardado (str): Ruta donde se guardará el archivo CSV resultante.
    Returns:
        None.
    """
    try:
        if os.path.isfile(ruta_archivo_json) and ruta_archivo_json.endswith('.json'):
            nombre_archivo = os.path.splitext(os.path.basename(ruta_archivo_json))[0]
            
            # Verificar si el nombre del archivo es diferente de "review"
            if nombre_archivo != "review":
                print("Leyendo el archivo:", ruta_archivo_json)
                with open(ruta_archivo_json, 'r', encoding='utf-8') as f:
                    data = [json.loads(line) for line in f]

                # Convertir la lista de diccionarios a DataFrame
                df = pd.DataFrame(data)

                # Guardar el DataFrame como CSV en la ruta especificada
                nombre_archivo_csv = os.path.join(ruta_guardado, nombre_archivo + '.csv')
                df.to_csv(nombre_archivo_csv, index=False)
                print("Se guardó con éxito en:", nombre_archivo_csv)
    except Exception as e:
        print(f"Error al procesar el archivo JSON: {e}")

def leer_contenido_carpeta(ruta_carpeta, ruta_guardado):
    try:
        # Verificar si la ruta es un directorio
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            contenido = os.listdir(ruta_carpeta)
            
            # Recorrer la lista de archivos y directorios
            for item in contenido:
                item_ruta = os.path.join(ruta_carpeta, item)
                
                # Si es un archivo, procesar si es JSON
                if os.path.isfile(item_ruta):
                    procesar_archivo_json(item_ruta, ruta_guardado)
                
                # Si es un directorio, llamar recursivamente a la función
                elif os.path.isdir(item_ruta):
                    print(f"Entrando en carpeta: {item_ruta}")
                    leer_contenido_carpeta(item_ruta, ruta_guardado)
        else:
            print(f"La ruta '{ruta_carpeta}' no es un directorio válido.")
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")

In [ ]:
ruta_carpeta = '../Data/Yelp/'
ruta_guardado = '../csv/Yelp/'  
leer_contenido_carpeta(ruta_carpeta, ruta_guardado)

### Yelp review devidido

In [ ]:
def dividir_archivo_json_en_csv(ruta_json_grande, carpeta_archivos_pequenos, tamano_archivo_pequeno=450000):
    
    """ 
    Dividir un archivo JSON grande en archivos CSV pequeños.

    Parámetros:
        ruta_json_grande (str): Ruta del archivo JSON grande.
        carpeta_archivos_pequenos (str): Ruta de la carpeta donde se guardarán los archivos CSV pequeños.
        tamano_archivo_pequeno (int): Tamaño máximo de cada archivo CSV pequeño (en líneas). Por defecto, 450000.

    Retorna:
        None. Los archivos CSV pequeños se guardan en la carpeta especificada.
    """
    # Función para escribir un fragmento JSON en un archivo CSV
    def escribir_fragmento_csv(numero_archivo, fragmento):
        nombre_archivo = f'fragmento_{numero_archivo}.csv'
        ruta_archivo = os.path.join(carpeta_archivos_pequenos, nombre_archivo)
        with open(ruta_archivo, 'w', newline='', encoding='utf-8') as archivo_csv:
            escritor_csv = csv.DictWriter(archivo_csv, fieldnames=fragmento[0].keys())
            escritor_csv.writeheader()
            escritor_csv.writerows(fragmento)

    # Leer el archivo JSON grande y dividirlo en fragmentos más pequeños
    with open(ruta_json_grande, 'r', encoding='utf-8') as archivo_grande:
        fragmento_actual = []
        numero_archivo = 1
        for linea in archivo_grande:
            objeto_json = json.loads(linea)
            fragmento_actual.append(objeto_json)
            if len(fragmento_actual) == tamano_archivo_pequeno:
                escribir_fragmento_csv(numero_archivo, fragmento_actual)
                numero_archivo += 1
                fragmento_actual = []
        # Escribir el último fragmento si es necesario
        if fragmento_actual:
            escribir_fragmento_csv(numero_archivo, fragmento_actual)

In [ ]:
ruta_json_grande = '../Data/Yelp/review.json'
carpeta_archivos_pequenos = '../csv/Yelp/review/'
dividir_archivo_json_en_csv(ruta_json_grande, carpeta_archivos_pequenos)

### Business

In [ ]:
# Lee el archivo pickle desde la ubicación especificada
buis = pd.read_pickle('../Data/Yelp/business.pkl')

# Guarda el DataFrame como un archivo CSV en la ubicación deseada
ruta_guardado = '../csv/Yelp/business.csv'
buis.to_csv(ruta_guardado, index=False)

Como primer paso, se llevó a cabo la carga de los datos en la plataforma en la nube Google Cloud Platform (GCP) mediante las funciones descritas anteriormente, lo que nos permitió tener acceso a los datos en Google BigQuery. Posteriormente, una vez definidos los objetivos y alcances del proyecto, procedimos a realizar una extracción de datos para trabajar con la información relevante para nuestro análisis.

En esta fase, se llevaron a cabo consultas en BigQuery, aprovechando la robustez de esta herramienta para establecer relaciones entre tablas. A través de estas consultas, logramos extraer únicamente la información que será crucial en las etapas posteriores del proyecto


A través de esta consulta en BigQuery, realizamos la fusión de la información de las tablas "business" y "review". También aplicamos filtros según la zona donde vamos a desarrollar el análisis y por fecha.

In [15]:
consulta_merge_businness_review = """
SELECT
  bussines.business_id,
  bussines.name,
  bussines.categories,
  bussines.city,
  bussines.latitude,
  bussines.longitude,
  bussines.stars AS bussines_stars,
  bussines.review_count,
  bussines.state,
  review.text,
  review.date,
  review.stars AS review_stars,
  review.cool,
  review.funny,
  review.useful,
  review.review_id
FROM
  `theta-byte-412400.YELP_REVIEW.RAW_REVIEW` AS review
JOIN
  `theta-byte-412400.YELP_BUSINESS.RAW_BUSINNES` AS bussines
ON
  review.business_id = bussines.business_id
WHERE
  DATE(review.date) >= '2021-01-01'
  AND bussines.categories LIKE '%Hotel%'
  AND bussines.state IN ('CA', 'OR', 'WA', 'AZ', 'NV');
"""

exportamos el resultado de la consulta en formato JSON para trabajar con estos datos

In [16]:
ruta_archivo_json = '../Data/bq-results-20240130-200859-1706645377030.json'

# Cargar el archivo JSON en un DataFrame de Pandas
df = pd.read_json(ruta_archivo_json, lines=True)

# Obtener información sobre el número de filas y columnas en el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4413 entries, 0 to 4412
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   business_id     4413 non-null   object             
 1   name            4413 non-null   object             
 2   categories      4413 non-null   object             
 3   city            4413 non-null   object             
 4   latitude        4413 non-null   float64            
 5   longitude       4413 non-null   float64            
 6   bussines_stars  4413 non-null   float64            
 7   review_count    4413 non-null   int64              
 8   state           4413 non-null   object             
 9   text            4413 non-null   object             
 10  date            4413 non-null   datetime64[ns, UTC]
 11  review_stars    4413 non-null   int64              
 12  cool            4413 non-null   int64              
 13  funny           4413 non-null   i

In [17]:
df.head()

,business_id,name,categories,city,latitude,longitude,bussines_stars,review_count,state,text,date,review_stars,cool,funny,useful,review_id
0,yi9udL5fcmBjwcBcygv51Q,Canal Place,"Shoe Stores, Fashion, Caterers, Shopping, Hote...",New Orleans,29.950999,-90.064996,3.5,112,CA,The Canal Place is full of upscale shopping. W...,2021-11-28 18:53:59+00:00,3,3,0,6,zOGcU9618uIIpFCurQz85g
1,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,We had a great experience with College Hunks r...,2021-01-18 03:09:14+00:00,5,0,0,1,13XkWHVhguwZWHIibguVEA
2,9NfPTgy_L9X57ep_Wq13Iw,Condor Express Whale Watching,"Whale Watching Tours, Hotels & Travel, Event P...",Santa Barbara,34.408290,-119.691469,4.5,82,AZ,"Honestly, deserves more than 5 stars. I had lo...",2021-06-29 03:26:50+00:00,5,0,0,2,gHpNqTAH2FpPCBO6p_YtYA
3,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,I contacted them today 10/2 for a move on 10/2...,2021-10-02 20:03:22+00:00,1,0,0,0,AWktCbWrHNaT9pMBcr5VcQ
4,UcUgyFueY2LyHe3Zhcuhkg,Best Western Suites Near Opryland,"Hotels, Event Planning & Services, Hotels & Tr...",Nashville,36.223499,-86.696996,2.5,48,NV,Stayed 10/13/2021 - 10/17/2021. Rooms 211 & 2...,2021-10-22 16:41:13+00:00,1,0,0,0,UFoXp5oL5P8iFAVnYuGejA


Como próximo paso, llevamos a cabo un análisis de sentimiento sobre las reseñas que los usuarios proporcionaron, con el objetivo de compararlo posteriormente con las calificaciones en estrellas que estos mismos usuarios otorgan al negocio. De esta manera, buscamos determinar cuál de estos criterios es más fidedigno para nuestra evaluación

In [19]:
# Función para asignar categorías personalizadas
def categorizar_sentimiento(polaridad):
    if polaridad > 0.5:
        return 'Excelente'
    elif 0.1 <= polaridad <= 0.5:
        return 'Bueno'
    elif -0.1 <= polaridad < 0.1:
        return 'Regular'
    elif -0.5 <= polaridad < -0.1:
        return 'Malo'
    else:
        return 'Muy Malo'

# Aplica el análisis de sentimiento solo para reseñas presentes
mask = df['text'].notnull()
df.loc[mask, 'sentiment_analysis'] = df.loc[mask, 'text'].apply(lambda x: categorizar_sentimiento(TextBlob(str(x)).sentiment.polarity))

df.head()

,business_id,name,categories,city,latitude,longitude,bussines_stars,review_count,state,text,date,review_stars,cool,funny,useful,review_id,sentiment_analysis
0,yi9udL5fcmBjwcBcygv51Q,Canal Place,"Shoe Stores, Fashion, Caterers, Shopping, Hote...",New Orleans,29.950999,-90.064996,3.5,112,CA,The Canal Place is full of upscale shopping. W...,2021-11-28 18:53:59+00:00,3,3,0,6,zOGcU9618uIIpFCurQz85g,Regular
1,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,We had a great experience with College Hunks r...,2021-01-18 03:09:14+00:00,5,0,0,1,13XkWHVhguwZWHIibguVEA,Regular
2,9NfPTgy_L9X57ep_Wq13Iw,Condor Express Whale Watching,"Whale Watching Tours, Hotels & Travel, Event P...",Santa Barbara,34.408290,-119.691469,4.5,82,AZ,"Honestly, deserves more than 5 stars. I had lo...",2021-06-29 03:26:50+00:00,5,0,0,2,gHpNqTAH2FpPCBO6p_YtYA,Bueno
3,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,I contacted them today 10/2 for a move on 10/2...,2021-10-02 20:03:22+00:00,1,0,0,0,AWktCbWrHNaT9pMBcr5VcQ,Regular
4,UcUgyFueY2LyHe3Zhcuhkg,Best Western Suites Near Opryland,"Hotels, Event Planning & Services, Hotels & Tr...",Nashville,36.223499,-86.696996,2.5,48,NV,Stayed 10/13/2021 - 10/17/2021. Rooms 211 & 2...,2021-10-22 16:41:13+00:00,1,0,0,0,UFoXp5oL5P8iFAVnYuGejA,Regular


Categorizamos la columna "review_stars" asignando las etiquetas de 1 como "Muy Malo", 2 como "Malo", 3 como "Regular", y 4 y 5 como "Bueno" y "Excelente", respectivamente. Posteriormente, comparamos esta clasificación con nuestro análisis de sentimiento, donde aplicamos también una categorización. En este caso, asignamos las siguientes etiquetas: 0 representa un comentario negativo, 1 un comentario neutro y 2 un comentario positivo.

In [21]:
# Define las categorías para las estrellas
star_mapping = {1: 'Muy Malo', 2: 'Malo', 3: 'Regular', 4: 'Bueno', 5: 'Excelente'}
df['stars_category'] = df['review_stars'].map(star_mapping)

In [22]:
# Selecciona las columnas que deseas mostrar
selected_columns = ['stars_category', 'sentiment_analysis','review_stars','text']

# Muestra las primeras 20 filas de las columnas seleccionadas
df[selected_columns].head(5)

,stars_category,sentiment_analysis,review_stars,text
0,Regular,Regular,3,The Canal Place is full of upscale shopping. W...
1,Excelente,Regular,5,We had a great experience with College Hunks r...
2,Excelente,Bueno,5,"Honestly, deserves more than 5 stars. I had lo..."
3,Muy Malo,Regular,1,I contacted them today 10/2 for a move on 10/2...
4,Muy Malo,Regular,1,Stayed 10/13/2021 - 10/17/2021. Rooms 211 & 2...


Como se observan rápidamente algunas discrepancias entre los dos análisis, creamos una tabla de contingencias para evaluarlas.

In [23]:
# Crea la tabla de contingencia
contingency_table = pd.crosstab(df['sentiment_analysis'], df['stars_category'], margins=True, margins_name='Total')

# Renombra la columna 'All' a 'Total'
contingency_table.rename(columns={'All': 'Total'}, inplace=True)

# Visualiza la tabla de contingencia
print(contingency_table)

stars_category      Bueno  Excelente  Malo  Muy Malo  Regular  Total
sentiment_analysis                                                  
Bueno                 434       1387   135       260      213   2429
Excelente              41        298     2         4        9    354
Malo                    5          1    55       389        9    459
Muy Malo                0          1     1        45        0     47
Regular                53         47   195       716      113   1124
Total                 533       1734   388      1414      344   4413


Podemos observar que hay un total de 945 coincidencias entre las dos columnas, de un total de 4413 filas. Se decide realizar una ponderación entre estas dos columnas para tomar en cuenta ambos análisis y llegar a un resultado más fidedigno.

se realiza un mapeo de las categorías de la columna "sentiment_analysis" a números del 1 al 5, asignándoles un valor numérico acorde a su grado de positividad. Luego, se asignan pesos a las métricas, especificando que la métrica de las estrellas de la revisión tiene un peso del 70%, mientras que el análisis de sentimiento tiene un peso del 30%. Finalmente, se calcula una nueva métrica combinada, donde se promedian ponderadamente los valores de las métricas de las estrellas y el análisis de sentimiento, obteniendo así una medida combinada que refleja tanto la evaluación de los usuarios como el análisis de sentimiento, considerando sus respectivos pesos. Este enfoque permite integrar ambas fuentes de información de manera equilibrada en una sola métrica para evaluar la satisfacción general

In [24]:
# Mapeo de categorías a números del 1 al 5
sentiment_mapping_numerico = {'Excelente': 5, 'Bueno': 4, 'Regular': 3, 'Malo': 2, 'Muy Malo': 1}

# Aplicar el mapeo a la columna sentiment_analysis
df['sentiment_analysis_numerico'] = df['sentiment_analysis'].map(sentiment_mapping_numerico)

# Asignar pesos a las métricas
peso_review_stars = 0.7
peso_analisis_sentimiento = 0.3

# Calcular métrica combinada
df['metrica_combinada'] = (peso_review_stars * df['review_stars'] +
                           peso_analisis_sentimiento * df['sentiment_analysis_numerico']) / (peso_review_stars + peso_analisis_sentimiento)

In [26]:
selected_columns = ['stars_category', 'sentiment_analysis','review_stars','metrica_combinada']

df[selected_columns].head(5)

,stars_category,sentiment_analysis,review_stars,metrica_combinada
0,Regular,Regular,3,3.0
1,Excelente,Regular,5,4.4
2,Excelente,Bueno,5,4.7
3,Muy Malo,Regular,1,1.6
4,Muy Malo,Regular,1,1.6
